In [13]:
import pandas as pd
import numpy as np

In [12]:
# user_df
u_cols = ['user_id','age','sex','occupation','zip_code']
users = pd.read_csv('/content/drive/MyDrive/recomm_sample/u.user.csv',sep='|',names=u_cols,encoding='latin_1')
users = users.set_index('user_id')
print(users.shape)
users.head()

(943, 4)


,age,sex,occupation,zip_code
user_id,,,,
1,24,M,technician,85711
2,53,F,other,94043
3,23,M,writer,32067
4,24,M,technician,43537
5,33,F,other,15213


In [11]:
# item_df
i_cols = ['movie_id','title','release date','video release date','IMDB URL','unknown','Action','Adventure','Animation','children\s',
          'comedy','crime','Documentary','Drama','Fantasy','Film-Noir','Horror','Musical','Mystery','Romance',
          'Sci-Fi','Thriller','war','western']

movies = pd.read_csv('/content/drive/MyDrive/recomm_sample/u.item.csv',sep='|',names=i_cols,
                     encoding='latin_1')
movies = movies.set_index('movie_id')
print(movies.shape)
movies.head()

(1682, 23)


,title,release date,video release date,IMDB URL,unknown,Action,Adventure,Animation,children\s,comedy,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,war,western
movie_id,,,,,,,,,,,,,,,,,,,,,
1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [10]:
# ratings_df
r_cols = ['user_id','movie_id','rating','timestamp']
ratings = pd.read_csv('/content/drive/MyDrive/recomm_sample/u.data.csv',sep='\t',names=r_cols,
                      encoding='latin_1')
ratings = ratings.set_index('user_id')
print(ratings.shape)
ratings.head()

(100000, 3)


,movie_id,rating,timestamp
user_id,,,
196,242,3,881250949
186,302,3,891717742
22,377,1,878887116
244,51,2,880606923
166,346,1,886397596


In [17]:
# movie 베스트 셀러 추천하기
# : 영화 평점을 평균한 후, 평균값이 가장 높은 제품을 추천

def recomm_movie_1(n_items):
  movie_sort = movie_mean.sort_values(ascending=False)[:n_items]    # 상위 n_items(개수)만큼의 평점 높은 영화 리턴
  recomm_movies = movies.loc[movie_sort.index]   # movie_df에서 movie_sort 영화 정보 가져옴
  recommendations = recomm_movies['title']
  return recommendations

movie_mean = ratings.groupby(['movie_id'])['rating'].mean()   # movie_id별로 rating값에 대해 평균
print(recomm_movie_1(5))

movie_id
814                         Great Day in Harlem, A (1994)
1599                        Someone Else's America (1995)
1201           Marlene Dietrich: Shadow and Light (1996) 
1122                       They Made Me a Criminal (1939)
1653    Entertaining Angels: The Dorothy Day Story (1996)
Name: title, dtype: object


In [36]:
# 추천시스템 모델(위 베스트 셀러 모델)의 정확도 계산
def RMSE(y_true,y_pred):
  return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

rmse = []

for user in set(ratings.index):          # ratings의 모든 user_id에 대해서 for문 수행
  y_true = ratings.loc[user]['rating']  # user 실제 평점
  y_pred = movie_mean[ratings.loc[user]['movie_id']]  # 각 영화의 평점 평균을 y_pred에 때려박음
  acc = RMSE(y_true,y_pred)
  rmse.append(acc)
print(np.mean(rmse))   # rmse 0.996

0.996007224010567
